# Final Project NLP Smart Scientific Content Auto-completion System


a text auto-complete system that supports scientists in writing articles in:
- completing some typed words/letters.
- Identifying words/phrases that are very unlikely to occur in some text contexts and suggesting proper replacements.


Karmel Salah
15/4/2022

# We will use Colab for a Free GPU so if you are using Colab frist connect to your drive to get the dataset


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#unzip the dataset folder
!unzip ./drive/MyDrive/dataset.zip 

Archive:  ./drive/MyDrive/dataset.zip
   creating: dataset/
  inflating: dataset/1012.2609.txt   
  inflating: dataset/1207.1847v1.txt  
  inflating: dataset/1209.3126v1.txt  
  inflating: dataset/1210.0852.txt   
  inflating: dataset/1305.6143.txt   
  inflating: dataset/1306.6802v2.txt  
  inflating: dataset/1308.0850v5.txt  
  inflating: dataset/1309.4168v1.txt  
  inflating: dataset/1312.3005v3.txt  
  inflating: dataset/1403.6652v2.txt  
  inflating: dataset/1404.3377v1.txt  
  inflating: dataset/1404.4326v1.txt  
  inflating: dataset/1405.4053v2.txt  
  inflating: dataset/1405.4273v1.txt  
  inflating: dataset/1406.1078v3.txt  
  inflating: dataset/1408.5882v2.txt  
  inflating: dataset/1409.0473v7.txt  
  inflating: dataset/1409.2329v5.txt  
  inflating: dataset/1409.3215v3.txt  
  inflating: dataset/1410.5329v4.txt  
  inflating: dataset/1410.8206v4.txt  
  inflating: dataset/1411.4555v2.txt  
  inflating: dataset/1412.1058v2.txt  
  inflating: dataset/1412.2007v2.txt  
  infla

# Importing Libraries

In [4]:
import pandas as pd
import numpy as np
import os, glob
import math
import matplotlib.pyplot as plt
import seaborn as sns
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [5]:
# get the names of each folders
files_names = os.listdir('./dataset')
len(files_names)

1064

# Convert to DataFrame

In [6]:
df = pd.DataFrame([], columns = ["id", "articles"])
print(f"data frame is {df}")

data frame is Empty DataFrame
Columns: [id, articles]
Index: []


In [7]:
def convert_to_df(df, files_names):
        id = []
        articles= []
        # Define files place and read each sub file for cleaning
        for index, file in enumerate(files_names):
                # read each article files
                path = os.getcwd()
                file_path = '{}/dataset/{}'.format(path,file)
                with open(file_path,'r', encoding='utf-16',  errors="ignore") as f:
                    texts = f.read() 
                    articles.append(texts) 
                    id.append(index)
                    f.close()
        print("the ids", id)
        print("the articles", articles)
        df['id'] = id
        df["articles"] = articles
        return df


In [8]:
df = convert_to_df(df, files_names)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
df["articles"]

0       Purely sequence-trained neural networks for AS...
1       Greedy Optimized Multileaving for Personalizat...
2       END-TO-END ATTENTION-BASED LARGE VOCABULARY SP...
3       Published as a conference paper at ICLR 2019\n...
4       Subword-level Word Vector Representations for ...
                              ...                        
1059    The Sample Complexity of Online One-Class Coll...
1060    DELTA\n\nA DEep learning based Language Techno...
1061    Generating Sentences from a Continuous Space\n...
1062    Published as a conference paper at ICLR 2019\n...
1063    Context-Sensitive Lexicon Features for Neural ...
Name: articles, Length: 1064, dtype: object

In [10]:
# count the number of words for each doc
df['word_count'] = df['articles'].apply(lambda x: len(str(x).split()))
df["word_count"]

0        4643
1        4206
2        6416
3        8190
4        6898
        ...  
1059    14577
1060     6459
1061     8798
1062     7224
1063     6704
Name: word_count, Length: 1064, dtype: int64

 # Text pre-processing:

In [11]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    # Remove leading and trailing spaces
    text= text.strip()  
    text= re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text= re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    # get ride of one letter word!
    text = re.sub(r'\b\w\b','',text) 
    
      
    return text

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [13]:
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [14]:
#STEMMING
# Initialize the stemming
sb = SnowballStemmer("english")

#Tokenize the text
def stemming(stripped):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[sb.stem(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [18]:
!apt-get install enchant

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 42 not upgraded.
Need to get 1,312 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-i

In [19]:
!pip3 install pyenchant

In [20]:
# Import the dict library
import enchant

In [21]:
#Remove non English words
dictionary = enchant.Dict("en_US")
   
def remove_non_english(text):
        en_words= []
        for word in text:
            if dictionary.check(word):
                en_words.append(word)
        text = ' '.join(str(e) for e in en_words)
        return text

In [22]:
df['english_text'] = df['articles'].apply(lambda text: remove_non_english(text.split()))

In [23]:
df['english_text'][0]

'Purely sequence-trained neural networks for based on lattice-free Daniel 1 2 1 Daniel 3 1 1 Na 4 Wang 1 1 2 1 Center for Language and Speech Processing 2 Human Language Technology Center of Excellence 3 Department of Computer Cornell University 4 Innovation and Intelligence Technology Ltd. The Johns Hopkins MD USA Abstract In this paper we describe a method to perform sequence- training of neural network acoustic models with- out the need for frame-level cross-entropy We use the lattice-free version of the maximum mutual information To make its computation feasible we use a phone n-gram language in place of the word language model. To further reduce its space and time com- we compute the objective function using neural network outputs at one third the standard frame rate. These changes en- able us to perform the computation for the forward-backward algorithm on Further the reduced output frame-rate also provides a speed-up during decoding. We present results on 5 different tasks with 

In [25]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [28]:
df['clean_text'] = df['english_text'].apply(lambda x: finalpreprocess(x))
df.head()

,id,articles,word_count,english_text,clean_text
0,0,Purely sequence-trained neural networks for AS...,4643,Purely sequence-trained neural networks for ba...,purely sequence train neural network base latt...
1,1,Greedy Optimized Multileaving for Personalizat...,4206,Greedy Optimized for Inc. Japan Inc. Japan Inc...,greedy optimize inc japan inc japan inc japan ...
2,2,END-TO-END ATTENTION-BASED LARGE VOCABULARY SP...,6416,END-TO-END ATTENTION-BASED LARGE VOCABULARY SP...,end end attention base large vocabulary speech...
3,3,Published as a conference paper at ICLR 2019\n...,8190,Published as a conference paper at 2019 V ON M...,publish conference paper rain language technol...
4,4,Subword-level Word Vector Representations for ...,6898,Word Vector Representations for Korean Park 1 ...,word vector representation korean park alice o...


In [29]:
articles = df['english_text']

In [30]:
articles[0]

'Purely sequence-trained neural networks for based on lattice-free Daniel 1 2 1 Daniel 3 1 1 Na 4 Wang 1 1 2 1 Center for Language and Speech Processing 2 Human Language Technology Center of Excellence 3 Department of Computer Cornell University 4 Innovation and Intelligence Technology Ltd. The Johns Hopkins MD USA Abstract In this paper we describe a method to perform sequence- training of neural network acoustic models with- out the need for frame-level cross-entropy We use the lattice-free version of the maximum mutual information To make its computation feasible we use a phone n-gram language in place of the word language model. To further reduce its space and time com- we compute the objective function using neural network outputs at one third the standard frame rate. These changes en- able us to perform the computation for the forward-backward algorithm on Further the reduced output frame-rate also provides a speed-up during decoding. We present results on 5 different tasks with 

In [31]:
### Tokinze Text to sentence form before cleaning!
def toknize_sent(article):
          sents = nltk.sent_tokenize(article)
          return sents
    
sentences = []
for article in articles:
    sentences.append(toknize_sent(article))
    
print(sentences[0])


['Purely sequence-trained neural networks for based on lattice-free Daniel 1 2 1 Daniel 3 1 1 Na 4 Wang 1 1 2 1 Center for Language and Speech Processing 2 Human Language Technology Center of Excellence 3 Department of Computer Cornell University 4 Innovation and Intelligence Technology Ltd.', 'The Johns Hopkins MD USA Abstract In this paper we describe a method to perform sequence- training of neural network acoustic models with- out the need for frame-level cross-entropy We use the lattice-free version of the maximum mutual information To make its computation feasible we use a phone n-gram language in place of the word language model.', 'To further reduce its space and time com- we compute the objective function using neural network outputs at one third the standard frame rate.', 'These changes en- able us to perform the computation for the forward-backward algorithm on Further the reduced output frame-rate also provides a speed-up during decoding.', 'We present results on 5 differen

In [32]:
df['sentecnes_tokens'] = sentences

In [33]:
df['sentecnes_tokens']

0       [Purely sequence-trained neural networks for b...
1       [Greedy Optimized for Inc. Japan Inc. Japan In...
2       [END-TO-END ATTENTION-BASED LARGE VOCABULARY S...
3       [Published as a conference paper at 2019 V ON ...
4       [Word Vector Representations for Korean Park 1...
                              ...                        
1059    [The Sample Complexity of Online One-Class Col...
1060    [DELTA A learning based Language Technology Hu...
1061    [Generating Sentences from a Continuous Space ...
1062    [Published as a conference paper at 2019 S N N...
1063    [Context-Sensitive Lexicon Features for Neural...
Name: sentecnes_tokens, Length: 1064, dtype: object

In [34]:
# save the dataframe into file
df.to_csv('dataframe.csv')  

# Vectorization

In [35]:
# Cleaning each sentence 
sent_cleaned_text = []

count = 0
for article in sentences:
    for i,sentence in enumerate(article):
        sent_cleaned_text.append(finalpreprocess(article[i]))


In [36]:
print(sent_cleaned_text[:2])

['purely sequence train neural network base lattice free daniel daniel na wang center language speech process human language technology center excellence department computer cornell university innovation intelligence technology ltd', 'john hopkins md usa abstract paper describe method perform sequence train neural network acoustic model need frame level cross entropy use lattice free version maximum mutual information make computation feasible use phone gram language place word language model']


In [37]:
# Initialize the Tokenizer class
tokenizer = Tokenizer()

# Generate the word index dictionary
tokenizer.fit_on_texts(sent_cleaned_text)

# Define the total words. You add 1 for the index `0` which is just the padding token.
total_words = len(tokenizer.word_index) + 1

print(f'word index dictionary: {tokenizer.word_index}')
print(f'total words: {total_words}')

word index dictionary: {'model': 1, 'use': 2, 'word': 3, 'learn': 4, 'network': 5, 'neural': 6, 'language': 7, 'base': 8, 'train': 9, 'data': 10, 'set': 11, 'method': 12, 'task': 13, 'result': 14, 'sentence': 15, 'system': 16, 'show': 17, 'layer': 18, 'feature': 19, 'one': 20, 'two': 21, 'text': 22, 'table': 23, 'also': 24, 'sequence': 25, 'vector': 26, 'performance': 27, 'representation': 28, 'work': 29, 'information': 30, 'machine': 31, 'user': 32, 'input': 33, 'number': 34, 'test': 35, 'training': 36, 'attention': 37, 'time': 38, 'page': 39, 'proceeding': 40, 'propose': 41, 'conference': 42, 'state': 43, 'figure': 44, 'large': 45, 'translation': 46, 'approach': 47, 'deep': 48, 'different': 49, 'give': 50, 'well': 51, 'score': 52, 'size': 53, 'output': 54, 'level': 55, 'document': 56, 'generate': 57, 'embed': 58, 'label': 59, 'function': 60, 'term': 61, 'parameter': 62, 'sample': 63, 'weight': 64, 'item': 65, 'learning': 66, 'high': 67, 'follow': 68, 'value': 69, 'experiment': 70, 'c

In [38]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Getting the X inputs and Y inputs through sequence of sentence
# Initialize the sequences list
input_sequences = []

# Loop over every line
for line in sent_cleaned_text:
    # Tokenize the current line
    token_list = tokenizer.texts_to_sequences([line])[0]

    # Loop over the line several times to generate the subphrases
    for i in range(1, len(token_list)):
        # Generate the subphrase
        n_gram_sequence = token_list[:i+1]
        # Append the subphrase to the sequences list
        input_sequences.append(n_gram_sequence)

In [ ]:
# Get the length of the longest line
max_sequence_len = max([len(x) for x in input_sequences])

In [ ]:
# Pad all sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
# Create inputs and label by splitting the last token in the subphrases
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

# Convert the label into one-hot arrays
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
# # Get sample sentence
sentence = corpus[0].split()
print(f'sample sentence: {sentence}')

# Initialize token list
token_list = []

# Look up the indices of each word and append to the list
for word in sentence: 
  token_list.append(tokenizer.word_index[word])

# Print the token list
print(token_list)

In [ ]:
# # Pick element
elem_number = 11

# Print token list and phrase
print(f'token list: {xs[elem_number]}')
print(f'decoded to text: {tokenizer.sequences_to_texts([xs[elem_number]])}')

# Print label
print(f'one-hot label: {ys[elem_number]}')
print(f'index of label: {np.argmax(ys[elem_number])}')

In [ ]:
# Hyperparameters
embedding_dim = 100
lstm_units = 150
learning_rate = 0.01

# Build the model
model = Sequential([
          Embedding(total_words, embedding_dim, input_length=max_sequence_len-1),
          Bidirectional(LSTM(lstm_units)),
          Dense(total_words, activation='softmax')
])

# Use categorical crossentropy because this is a multi-class problem
model.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
    metrics=['accuracy']
    )

# Print the model summary
model.summary()

In [ ]:
epochs = 100

# Train the model
history = model.fit(xs, ys, epochs=epochs)

In [ ]:
import matplotlib.pyplot as plt

# Plot utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

# Visualize the accuracy
plot_graphs(history, 'accuracy')

In [ ]:
# Define seed text
seed_text = "help me obi-wan kinobi youre my only hope"

# Define total words to predict
next_words = 100

# Loop until desired length is reached
for _ in range(next_words):

	# Convert the seed text to a token sequence
	token_list = tokenizer.texts_to_sequences([seed_text])[0]

	# Pad the sequence
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	
	# Feed to the model and get the probabilities for each index
	probabilities = model.predict(token_list)

	# Get the index with the highest probability
	predicted = np.argmax(probabilities, axis=-1)[0]

	# Ignore if index is 0 because that is just the padding.
	if predicted != 0:
		
		# Look up the word associated with the index. 
		output_word = tokenizer.index_word[predicted]

		# Combine with the seed text
		seed_text += " " + output_word

# Print the result	
print(seed_text)

In [ ]:
# Define seed text
seed_text = "help me obi-wan kinobi youre my only hope"

# Define total words to predict
next_words = 100

# Loop until desired length is reached
for _ in range(next_words):

	# Convert the seed text to a token sequence
  token_list = tokenizer.texts_to_sequences([seed_text])[0]

	# Pad the sequence
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	
	# Feed to the model and get the probabilities for each index
  probabilities = model.predict(token_list)

  # Pick a random number from [1,2,3]
  choice = np.random.choice([1,2,3])
	
  # Sort the probabilities in ascending order 
  # and get the random choice from the end of the array
  predicted = np.argsort(probabilities)[0][-choice]

	# Ignore if index is 0 because that is just the padding.
  if predicted != 0:
		
		# Look up the word associated with the index. 
	  output_word = tokenizer.index_word[predicted]

		# Combine with the seed text
	  seed_text += " " + output_word

# Print the result	
print(seed_text)